### SageMaker Large Scale prediction

In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


In [2]:
version_tag="202105222201"
pytorch_custom_image_name=f"ppi:cpu-{version_tag}"
instance_type = "ml.m5.large" 

In [3]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

In [4]:
bucket = "aegovan-data"

In [5]:
testfile= "s3://{}/processed_dataset/test_multiclass.json".format(bucket)
valfile="s3://{}/processed_dataset/val_multiclass.json".format(bucket)
trainfile= "s3://{}/processed_dataset/train_multiclass.json".format(bucket)



### Step 1: Convert pubtator format to inference json

The input pubtator files look like this.. These are converted to produce inference 

```text
20791654|a|Liver scan characteristics and liver function tests of 72 patients with proved hepatic malignancy (54 metastatic, 18 primary) were evaluated. Well-defined focal defects were observed in 83% of patients with metastatic and 77% of patients with primary liver carcinoma. In 10% of the patients with metastatic liver disease the distribution of radioactivity was normal. Four or more biochemical liver function tests were normal in 33% of metastatic and 29% of primary liver cancer patients. Hepatic enlargement was present in the scan in 94% of the patients with liver metastases; however, data obtained from 104 necropsies of patients with hepatic metastases showed that only 46% had hepatomegaly. We recommend, therefore, that a liver scan should be performed before major tumour surgery in every patient with known malignancy regardless of normal liver size or normal liver function tests.
20791654	58	66	patients	Species	9606
20791654	193	201	patients	Species	9606
20791654	229	237	patients	Species	9606
20791654	282	290	patients	Species	9606
20791654	478	486	patients	Species	9606
20791654	546	554	patients	Species	9606
20791654	624	632	patients	Species	9606
20791654	796	803	patient	Species	9606

20791817|a|5-Aminosalicylic acid given to rats as a single intravenous injection led to necrosis of the proximal convoluted tubules and of the renal papilla. These two lesions developed at the same time and the cortical lesions did not appear to be a consequence of the renal papillary necrosis. Since the compound possesses the molecular structure both of a phenacetin derivative and of a salicylate these observations may be relevant to the problem of renal damage incident to abuse of analgesic compounds and suggest the possibility that in this syndrome cortical lesions may develop independently of renal papillary necrosis.
20791817	31	35	rats	Species	10116

```

In [6]:
import datetime
date_fmt = datetime.datetime.today().strftime("%Y%m%d%H")

In [7]:
#s3_input_pubtator = "s3://aegovan-data/pubmed_json_parts_annotation_iseries/pubmed19n0550.json.txt"
s3_input_pubtator = "s3://aegovan-data/pubmed_json_parts_annotation_iseries/"
s3_id_mapping_file="s3://aegovan-data/settings/HUMAN_9606_idmapping.dat"

s3_output_pubmed_asbtract = f"s3://aegovan-data/pubmed_asbtract/inference_multi_{date_fmt}/"

In [8]:
# from sagemaker.network import NetworkConfig
# from sagemaker.processing import ProcessingInput, ProcessingOutput
# from sagemaker.processing import ScriptProcessor

# script_processor = ScriptProcessor(image_uri=docker_repo,
#                                        command=["python"],
#                                        env={'mode': 'python', 'PYTHONPATH':'/opt/ml/code'},
#                                        role=role,
#                                        instance_type=instance_type,
#                                        instance_count=10,
#                                        max_runtime_in_seconds=172800,
#                                        volume_size_in_gb = 50,
#                                        network_config=NetworkConfig(enable_network_isolation=False),
#                                        base_job_name ="ppi-large-inference-data-prep"


#                                        )


# sm_local_input_pubtator_txt = "/opt/ml/processing/input/data/json"
# sm_local_input_idmapping = "/opt/ml/processing/input/data/mapping"
# sm_local_output = "/opt/ml/processing/output"


# script_processor.run(
#         code='source/datatransformer/pubtator_annotations_inference_transformer.py',

#         arguments=[
        
#             sm_local_input_pubtator_txt,
#             sm_local_output,
#            "{}/{}".format(sm_local_input_idmapping,s3_id_mapping_file.split("/")[-1]) 

#         ],
    
#        inputs=[
#                 ProcessingInput(
#                     source=s3_input_pubtator,
#                     destination=sm_local_input_pubtator_txt,
#                     s3_data_distribution_type="ShardedByS3Key")

#             ,ProcessingInput(
#                     source=s3_id_mapping_file,
#                     destination=sm_local_input_idmapping,
#                     s3_data_distribution_type="FullyReplicated")
#             ],

#         outputs=[ProcessingOutput(
#                 source=sm_local_output, 
#                 destination=s3_output_pubmed_asbtract,
#                 output_name='inferenceabstracts')]
#    )

## Step 2: Run predictions

In [9]:
prepare_models = False

In [10]:
jobs_f1 = [
    
"ppimulticlass-bert-f1-2021-05-11-03-51-32-646",
"ppimulticlass-bert-f1-2021-05-11-03-51-18-103",
"ppimulticlass-bert-f1-2021-05-11-03-51-11-189",
"ppimulticlass-bert-f1-2021-05-10-19-31-36-944",
"ppimulticlass-bert-f1-2021-05-10-19-31-18-988",
"ppimulticlass-bert-f1-2021-05-10-16-23-25-703",
"ppimulticlass-bert-f1-2021-05-10-16-23-19-189",
"ppimulticlass-bert-f1-2021-05-10-16-23-03-547",
"ppimulticlass-bert-f1-2021-05-10-16-22-33-226"
]

In [11]:
jobs_loss = [
   
"ppimulticlass-bert-2021-05-08-20-29-59-117",
"ppimulticlass-bert-2021-05-08-20-29-39-694",
"ppimulticlass-bert-2021-05-08-20-29-11-549",
"ppimulticlass-bert-2021-05-08-20-29-06-842",
"ppimulticlass-bert-2021-05-08-19-21-50-296",
"ppimulticlass-bert-2021-05-08-19-21-43-271",
"ppimulticlass-bert-2021-05-08-19-21-15-995",
"ppimulticlass-bert-2021-05-08-19-08-50-131",
"ppimulticlass-bert-2021-05-08-17-12-32-759",
"ppimulticlass-bert-2021-05-08-17-11-49-920"

]



In [12]:
jobs=jobs_f1

In [13]:
s3_model_path_format = "s3://aegovan-data/ppi_multiclass_sagemakerresults/{}/output/model.tar.gz"

s3_model_paths = [s3_model_path_format.format(j) for j in jobs]

In [14]:
job_prefix = "{}-{}".format(jobs[-1][:32], len(jobs) )
s3_output_ensemble_models = "s3://aegovan-data/ppi_multiclass_ensemble_models/{}".format(job_prefix)

### Prepare ensemble models
TODO: This is just a hack to untar a bunch of zipped models and upload them to a single s3 locaton. Have a single processing job to do this is an overkill...

In [15]:
def get_processing_inputs_s3_local_path(s3_model_paths, sm_local_input):
    # Map the s3 model path to local input path
    inputs = []
    for i, s3_path in enumerate(s3_model_paths):
         p = ProcessingInput(
                        source=s3_path,
                        destination="{}/{}".format(sm_local_input.rstrip("/"), i)
         )
         inputs.append(p)
    return inputs


In [16]:
from sagemaker.network import NetworkConfig
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor


sm_local_input = "/opt/ml/processing/input/models"
sm_local_output = "/opt/ml/processing/output"

script_processor = ScriptProcessor(image_uri=docker_repo,
                                       command=["python"],
                                       env={'mode': 'python', 'PYTHONPATH':'/opt/ml/code'},
                                       role=role,
                                       instance_type=instance_type,
                                       instance_count=1,
                                       max_runtime_in_seconds=172800,
                                       volume_size_in_gb = 50,
                                       network_config=NetworkConfig(enable_network_isolation=False),
                                       base_job_name ="ppi-ensemble-model-packer"
                                       )


In [17]:

if prepare_models:
    # Work around to get over the processing job input limit size
    chunk_size=5
    for i in range(0, len(s3_model_paths), chunk_size ):

        script_processor.run(
                code='../src/inference/ensemble_inference_prepare_models.py',

                arguments=[
                    "--input-dir",
                    sm_local_input,
                    "--dest-dir",
                    sm_local_output

                ],

                inputs=get_processing_inputs_s3_local_path(s3_model_paths[i:i+chunk_size], sm_local_input),


                outputs=[ProcessingOutput(
                        source=sm_local_output, 
                        destination=s3_output_ensemble_models,
                        output_name='models')]
            )



### Run ensemble prediction

In [27]:
s3_output_predictions = "s3://aegovan-data/pubmed_asbtract/predictions_valtest_{}_{}/".format(job_prefix,date_fmt)
s3_input_data = trainfile

s3_input_models = s3_output_ensemble_models
s3_input_vocab = "s3://{}/embeddings/bert/".format(bucket)

In [28]:
pytorch_custom_image_name=f"ppi:gpu-{version_tag}"


instance_type = "ml.p3.2xlarge" 
instance_count = 1

In [20]:
#temp
#s3_output_pubmed_asbtract = f"s3://aegovan-data/pubmed_asbtract/inference_multi_2020123123/"

In [29]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

In [30]:
from sagemaker.network import NetworkConfig
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(image_uri=docker_repo,
                                       command=["python"],
                                       env={'mode': 'python', 'PYTHONPATH':'/opt/ml/code'},
                                       role=role,
                                       instance_type=instance_type,
                                       instance_count=instance_count,
                                       max_runtime_in_seconds=15 * 60,
                                       volume_size_in_gb = 250,
                                       network_config=NetworkConfig(enable_network_isolation=False),
                                       base_job_name ="ppi-ensemble-inference"
                                       )


sm_local_input_models = "/opt/ml/processing/input/data/models"
sm_local_input_data = "/opt/ml/processing/input/data/jsonlines"
sm_local_input_vocab = "/opt/ml/processing/input/data/vocab"


sm_local_output = "/opt/ml/processing/output"



script_processor.run(
        code='../src/inference/ppi_multiclass_batch_predict.py',

        arguments=[
            sm_local_input_data,
            sm_local_input_models,
            sm_local_output,
            "--ensemble", "1",
            "--tokenisor_data_dir", sm_local_input_vocab,
            "--protein_name_replacer_random_seed", "43"
        ],

        inputs=[
                ProcessingInput(
                    source=s3_input_data,
                    destination=sm_local_input_data,
                    s3_data_distribution_type="ShardedByS3Key"),

            ProcessingInput(
                    source=s3_input_models,
                    destination=sm_local_input_models,
                    s3_data_distribution_type="FullyReplicated"),
            
            ProcessingInput(
                    source=s3_input_vocab,
                    destination=sm_local_input_vocab,
                    s3_data_distribution_type="FullyReplicated")
            ],


        outputs=[ProcessingOutput(
                source=sm_local_output, 
                destination=s3_output_predictions,
                output_name='predictions')]
    )





Job Name:  ppi-ensemble-inference-2021-05-29-18-11-41-294
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aegovan-data/processed_dataset/train_multiclass.json', 'LocalPath': '/opt/ml/processing/input/data/jsonlines', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aegovan-data/ppi_multiclass_ensemble_models/ppimulticlass-bert-f1-2021-05-10-9', 'LocalPath': '/opt/ml/processing/input/data/models', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-3', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aegovan-data/embeddings/bert/', 'LocalPath': '/opt/ml/processing/input/data/vocab', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputNa

2021-05-29 18:17:42,217 - inference.predictor - INFO - Completed cpu cuda:0 33
2021-05-29 18:17:42,218 - inference.predictor - INFO - running batch 34
2021-05-29 18:17:42,219 - inference.predictor - INFO - predict batch cuda:0 34
2021-05-29 18:17:42,243 - inference.predictor - INFO - copy cpu cuda:0 34
2021-05-29 18:17:42,516 - inference.predictor - INFO - Completed cpu cuda:0 34
2021-05-29 18:17:42,517 - inference.predictor - INFO - running batch 35
2021-05-29 18:17:42,518 - inference.predictor - INFO - predict batch cuda:0 35
2021-05-29 18:17:42,542 - inference.predictor - INFO - copy cpu cuda:0 35
2021-05-29 18:17:42,815 - inference.predictor - INFO - Completed cpu cuda:0 35
2021-05-29 18:17:42,817 - inference.predictor - INFO - running batch 36
2021-05-29 18:17:42,817 - inference.predictor - INFO - predict batch cuda:0 36
2021-05-29 18:17:42,842 - inference.predictor - INFO - copy cpu cuda:0 36
2021-05-29 18:17:43,115 - inference.predictor - INFO - Completed cpu cuda:0 36
2021-05-2

2021-05-29 18:17:52,188 - inference.predictor - INFO - Completed cpu cuda:0 21
2021-05-29 18:17:52,190 - inference.predictor - INFO - running batch 22
2021-05-29 18:17:52,190 - inference.predictor - INFO - predict batch cuda:0 22
2021-05-29 18:17:52,215 - inference.predictor - INFO - copy cpu cuda:0 22
2021-05-29 18:17:52,488 - inference.predictor - INFO - Completed cpu cuda:0 22
2021-05-29 18:17:52,490 - inference.predictor - INFO - running batch 23
2021-05-29 18:17:52,490 - inference.predictor - INFO - predict batch cuda:0 23
2021-05-29 18:17:52,515 - inference.predictor - INFO - copy cpu cuda:0 23
2021-05-29 18:17:52,788 - inference.predictor - INFO - Completed cpu cuda:0 23
2021-05-29 18:17:52,789 - inference.predictor - INFO - running batch 24
2021-05-29 18:17:52,790 - inference.predictor - INFO - predict batch cuda:0 24
2021-05-29 18:17:52,815 - inference.predictor - INFO - copy cpu cuda:0 24
2021-05-29 18:17:53,088 - inference.predictor - INFO - Completed cpu cuda:0 24
2021-05-2

2021-05-29 18:18:02,272 - inference.predictor - INFO - Completed cpu cuda:0 10
2021-05-29 18:18:02,274 - inference.predictor - INFO - running batch 11
2021-05-29 18:18:02,274 - inference.predictor - INFO - predict batch cuda:0 11
2021-05-29 18:18:02,300 - inference.predictor - INFO - copy cpu cuda:0 11
2021-05-29 18:18:02,573 - inference.predictor - INFO - Completed cpu cuda:0 11
2021-05-29 18:18:02,573 - inference.predictor - INFO - running batch 12
2021-05-29 18:18:02,573 - inference.predictor - INFO - predict batch cuda:0 12
2021-05-29 18:18:02,599 - inference.predictor - INFO - copy cpu cuda:0 12
2021-05-29 18:18:02,872 - inference.predictor - INFO - Completed cpu cuda:0 12
2021-05-29 18:18:02,874 - inference.predictor - INFO - running batch 13
2021-05-29 18:18:02,874 - inference.predictor - INFO - predict batch cuda:0 13
2021-05-29 18:18:02,899 - inference.predictor - INFO - copy cpu cuda:0 13
2021-05-29 18:18:03,172 - inference.predictor - INFO - Completed cpu cuda:0 13
2021-05-2

2021-05-29 18:18:12,460 - inference.predictor - INFO - running batch 0
2021-05-29 18:18:12,461 - inference.predictor - INFO - predict batch cuda:0 0
2021-05-29 18:18:12,498 - inference.predictor - INFO - copy cpu cuda:0 0
2021-05-29 18:18:12,760 - inference.predictor - INFO - Completed cpu cuda:0 0
2021-05-29 18:18:12,763 - inference.predictor - INFO - running batch 1
2021-05-29 18:18:12,764 - inference.predictor - INFO - predict batch cuda:0 1
2021-05-29 18:18:12,789 - inference.predictor - INFO - copy cpu cuda:0 1
2021-05-29 18:18:13,062 - inference.predictor - INFO - Completed cpu cuda:0 1
2021-05-29 18:18:13,062 - inference.predictor - INFO - running batch 2
2021-05-29 18:18:13,063 - inference.predictor - INFO - predict batch cuda:0 2
2021-05-29 18:18:13,089 - inference.predictor - INFO - copy cpu cuda:0 2
2021-05-29 18:18:13,362 - inference.predictor - INFO - Completed cpu cuda:0 2
2021-05-29 18:18:13,362 - inference.predictor - INFO - running batch 3
2021-05-29 18:18:13,362 - inf

...

2021-05-29 18:18:47,261 - inference.predictor - INFO - Completed cpu cuda:0 25
2021-05-29 18:18:47,262 - inference.predictor - INFO - running batch 26
2021-05-29 18:18:47,262 - inference.predictor - INFO - predict batch cuda:0 26
2021-05-29 18:18:47,289 - inference.predictor - INFO - copy cpu cuda:0 26
2021-05-29 18:18:47,562 - inference.predictor - INFO - Completed cpu cuda:0 26
2021-05-29 18:18:47,564 - inference.predictor - INFO - running batch 27
2021-05-29 18:18:47,564 - inference.predictor - INFO - predict batch cuda:0 27
2021-05-29 18:18:47,591 - inference.predictor - INFO - copy cpu cuda:0 27
2021-05-29 18:18:47,865 - inference.predictor - INFO - Completed cpu cuda:0 27
2021-05-29 18:18:47,867 - inference.predictor - INFO - running batch 28
2021-05-29 18:18:47,867 - inference.predictor - INFO - predict batch cuda:0 28
2021-05-29 18:18:47,894 - inference.predictor - INFO - copy cpu cuda:0 28
2021-05-29 18:18:48,168 - inference.predictor - INFO - Completed cpu cuda:0 28
2021-05-2

2021-05-29 18:18:57,352 - inference.predictor - INFO - Completed cpu cuda:0 10
2021-05-29 18:18:57,354 - inference.predictor - INFO - running batch 11
2021-05-29 18:18:57,354 - inference.predictor - INFO - predict batch cuda:0 11
2021-05-29 18:18:57,379 - inference.predictor - INFO - copy cpu cuda:0 11
2021-05-29 18:18:57,655 - inference.predictor - INFO - Completed cpu cuda:0 11
2021-05-29 18:18:57,657 - inference.predictor - INFO - running batch 12
2021-05-29 18:18:57,657 - inference.predictor - INFO - predict batch cuda:0 12
2021-05-29 18:18:57,682 - inference.predictor - INFO - copy cpu cuda:0 12
2021-05-29 18:18:57,958 - inference.predictor - INFO - Completed cpu cuda:0 12
2021-05-29 18:18:57,958 - inference.predictor - INFO - running batch 13
2021-05-29 18:18:57,958 - inference.predictor - INFO - predict batch cuda:0 13
2021-05-29 18:18:57,984 - inference.predictor - INFO - copy cpu cuda:0 13
2021-05-29 18:18:58,259 - inference.predictor - INFO - Completed cpu cuda:0 13
2021-05-2

2021-05-29 18:19:07,648 - inference.predictor - INFO - running batch 0
2021-05-29 18:19:07,648 - inference.predictor - INFO - predict batch cuda:0 0
2021-05-29 18:19:07,695 - inference.predictor - INFO - copy cpu cuda:0 0
2021-05-29 18:19:07,951 - inference.predictor - INFO - Completed cpu cuda:0 0
2021-05-29 18:19:07,951 - inference.predictor - INFO - running batch 1
2021-05-29 18:19:07,951 - inference.predictor - INFO - predict batch cuda:0 1
2021-05-29 18:19:07,986 - inference.predictor - INFO - copy cpu cuda:0 1
2021-05-29 18:19:08,253 - inference.predictor - INFO - Completed cpu cuda:0 1
2021-05-29 18:19:08,257 - inference.predictor - INFO - running batch 2
2021-05-29 18:19:08,257 - inference.predictor - INFO - predict batch cuda:0 2
2021-05-29 18:19:08,283 - inference.predictor - INFO - copy cpu cuda:0 2
2021-05-29 18:19:08,558 - inference.predictor - INFO - Completed cpu cuda:0 2
2021-05-29 18:19:08,558 - inference.predictor - INFO - running batch 3
2021-05-29 18:19:08,559 - inf

2021-05-29 18:19:22,419 - inference.predictor - INFO - Completed cpu cuda:0 3
2021-05-29 18:19:22,421 - inference.predictor - INFO - running batch 4
2021-05-29 18:19:22,421 - inference.predictor - INFO - predict batch cuda:0 4
2021-05-29 18:19:22,448 - inference.predictor - INFO - copy cpu cuda:0 4
2021-05-29 18:19:22,723 - inference.predictor - INFO - Completed cpu cuda:0 4
2021-05-29 18:19:22,725 - inference.predictor - INFO - running batch 5
2021-05-29 18:19:22,725 - inference.predictor - INFO - predict batch cuda:0 5
2021-05-29 18:19:22,752 - inference.predictor - INFO - copy cpu cuda:0 5
2021-05-29 18:19:23,027 - inference.predictor - INFO - Completed cpu cuda:0 5
2021-05-29 18:19:23,028 - inference.predictor - INFO - running batch 6
2021-05-29 18:19:23,028 - inference.predictor - INFO - predict batch cuda:0 6
2021-05-29 18:19:23,054 - inference.predictor - INFO - copy cpu cuda:0 6
2021-05-29 18:19:23,330 - inference.predictor - INFO - Completed cpu cuda:0 6
2021-05-29 18:19:23,33


2021-05-29 18:19:32,442 - inference.predictor - INFO - Completed cpu cuda:0 36
2021-05-29 18:19:32,444 - inference.predictor - INFO - running batch 37
2021-05-29 18:19:32,444 - inference.predictor - INFO - predict batch cuda:0 37
2021-05-29 18:19:32,469 - inference.predictor - INFO - copy cpu cuda:0 37
2021-05-29 18:19:32,745 - inference.predictor - INFO - Completed cpu cuda:0 37
2021-05-29 18:19:32,747 - inference.predictor - INFO - running batch 38
2021-05-29 18:19:32,747 - inference.predictor - INFO - predict batch cuda:0 38
2021-05-29 18:19:32,772 - inference.predictor - INFO - copy cpu cuda:0 38
2021-05-29 18:19:33,049 - inference.predictor - INFO - Completed cpu cuda:0 38
2021-05-29 18:19:33,051 - inference.predictor - INFO - running batch 39
2021-05-29 18:19:33,051 - inference.predictor - INFO - predict batch cuda:0 39
2021-05-29 18:19:33,076 - inference.predictor - INFO - copy cpu cuda:0 39
2021-05-29 18:19:33,353 - inference.predictor - INFO - Completed cpu cuda:0 39
2021-05-